In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch_numopt
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
class Net(nn.Module):
    def __init__(self, input_size, device="cpu"):
        super().__init__()
        self.f1 = nn.Linear(input_size, 10, device=device)
        self.f2 = nn.Linear(10, 20, device=device)
        self.f3 = nn.Linear(20, 20, device=device)
        self.f4 = nn.Linear(20, 10, device=device)
        self.f5 = nn.Linear(10, 1, device=device)

        self.activation = nn.ReLU()
        # self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.activation(self.f1(x))
        x = self.activation(self.f2(x))
        x = self.activation(self.f3(x))
        x = self.activation(self.f4(x))
        x = self.f5(x)

        return x

In [4]:
# X, y = load_diabetes(return_X_y=True, scaled=False)
# X, y = make_regression(n_samples=1000, n_features=100)
X, y = make_friedman1(n_samples=1000, noise=1e-2)

X_scaler = MinMaxScaler()
X = X_scaler.fit_transform(X)
X = torch.Tensor(X).to(device)

y_scaler = MinMaxScaler()
y = y_scaler.fit_transform(y.reshape((-1, 1)))
y = torch.Tensor(y).to(device)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

torch_data = TensorDataset(X_train, y_train)
data_loader = DataLoader(torch_data, batch_size=1000)

torch.Size([800, 10]) torch.Size([800, 1])
torch.Size([200, 10]) torch.Size([200, 1])


In [5]:
model = Net(input_size=X.shape[1], device=device)
loss_fn = nn.MSELoss()

# opt = torch_numopt.ConjugateGradient(model=model, lr=1e-4, line_search_method="const", cg_method="PR")
opt = torch_numopt.ConjugateGradient(model=model, lr=10, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="armijo", cg_method="PR")
# opt = torch_numopt.ConjugateGradient(model=model, lr=10, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="wolfe", cg_method="PR")
# opt = torch_numopt.ConjugateGradient(model=model, lr=10, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="strong-wolfe", cg_method="PR")
# opt = torch_numopt.ConjugateGradient(model=model, lr=10, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="goldstein", cg_method="PR")


all_loss = {}
for epoch in range(1000):
    print("epoch: ", epoch, end="")
    all_loss[epoch + 1] = 0
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)

        all_loss[epoch + 1] += loss
    all_loss[epoch + 1] /= len(data_loader)
    print(", loss: {}".format(all_loss[epoch + 1].cpu().detach().numpy().item()))

epoch:  0, loss: 0.21060918271541595
epoch:  1, loss: 0.12482902407646179
epoch:  2, loss: 0.07619329541921616
epoch:  3, loss: 0.05481067672371864
epoch:  4, loss: 0.04528016597032547
epoch:  5, loss: 0.040952809154987335
epoch:  6, loss: 0.038965556770563126
epoch:  7, loss: 0.038045287132263184
epoch:  8, loss: 0.03761456534266472
epoch:  9, loss: 0.037409476935863495
epoch:  10, loss: 0.037308596074581146
epoch:  11, loss: 0.03725593164563179
epoch:  12, loss: 0.037225689738988876
epoch:  13, loss: 0.0371784046292305
epoch:  14, loss: 0.0371784046292305
epoch:  15, loss: 0.037133198231458664
epoch:  16, loss: 0.03710085153579712
epoch:  17, loss: 0.03707220405340195
epoch:  18, loss: 0.03707220405340195
epoch:  19, loss: 0.037019193172454834
epoch:  20, loss: 0.036981821060180664
epoch:  21, loss: 0.03696424514055252
epoch:  22, loss: 0.03696424514055252
epoch:  23, loss: 0.03690384328365326
epoch:  24, loss: 0.03686201944947243
epoch:  25, loss: 0.03683687746524811
epoch:  26, los

In [7]:
pred_train = model.forward(X_train).cpu().detach()
pred_test = model.forward(X_test).cpu().detach()
print(f"Train metrics: R2 = {r2_score(pred_train, y_train.cpu())}")
print(f"Test metrics:  R2 = {r2_score(pred_test, y_test.cpu())}")

Train metrics: R2 = 0.7564386457689405
Test metrics:  R2 = 0.7523754777767294


In [12]:
model = Net(input_size=X.shape[1], device=device)
loss_fn = nn.MSELoss()

# opt = torch_numopt.ConjugateGradient(model=model, lr=1e-4, line_search_method="const", cg_method="FR")
opt = torch_numopt.ConjugateGradient(model=model, lr=10, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="armijo", cg_method="DY")
# opt = torch_numopt.ConjugateGradient(model=model, lr=10, c1=0.1, c2=0.9, tau=0.1, line_search_method="backtrack", line_search_cond="wolfe", cg_method="FR")
# opt = torch_numopt.ConjugateGradient(model=model, lr=10, c1=0.1, c2=0.9, tau=0.1, line_search_method="backtrack", line_search_cond="strong-wolfe", cg_method="FR")
# opt = torch_numopt.ConjugateGradient(model=model, lr=10, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="goldstein", cg_method="FR")

all_loss = {}
for epoch in range(1000):
    print("epoch: ", epoch, end="")
    all_loss[epoch + 1] = 0
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)

        all_loss[epoch + 1] += loss
    all_loss[epoch + 1] /= len(data_loader)
    print(", loss: {}".format(all_loss[epoch + 1].cpu().detach().numpy().item()))

epoch:  0, loss: 0.7086195945739746
epoch:  1, loss: 0.4349656105041504
epoch:  2, loss: 0.4349656105041504
epoch:  3, loss: 0.4351779520511627
epoch:  4, loss: 0.05551861599087715
epoch:  5, loss: 0.047637324780225754
epoch:  6, loss: 0.047637324780225754
epoch:  7, loss: 0.04303358867764473
epoch:  8, loss: 0.04303358867764473
epoch:  9, loss: 0.0430341437458992
epoch:  10, loss: 0.040385108441114426
epoch:  11, loss: 0.040385108441114426
epoch:  12, loss: 0.04107861965894699
epoch:  13, loss: 0.03822510316967964
epoch:  14, loss: 0.03767276555299759
epoch:  15, loss: 0.03767276555299759
epoch:  16, loss: 0.037673261016607285
epoch:  17, loss: 0.036949459463357925
epoch:  18, loss: 0.03691224753856659
epoch:  19, loss: 0.03691224753856659
epoch:  20, loss: 0.03691224753856659
epoch:  21, loss: 0.03691224753856659
epoch:  22, loss: 0.03691227734088898
epoch:  23, loss: 0.036888718605041504
epoch:  24, loss: 0.036888718605041504
epoch:  25, loss: 0.036852262914180756
epoch:  26, loss: 

In [13]:
pred_train = model.forward(X_train).cpu().detach()
pred_test = model.forward(X_test).cpu().detach()
print(f"Train metrics: R2 = {r2_score(pred_train, y_train.cpu())}")
print(f"Test metrics:  R2 = {r2_score(pred_test, y_test.cpu())}")

Train metrics: R2 = 0.7406362185206627
Test metrics:  R2 = 0.7195633889184088
